In [18]:
import spacy
nlp= spacy.load('en_core_web_sm')

In [19]:
doc= nlp(u"The drunk nigger drove his car into a crowd and killed 4 people")

In [20]:
print(doc.text)

The drunk nigger drove his car into a crowd and killed 4 people


In [21]:
for tok in doc:
    print(f'{tok.text:{20}}----{tok.pos_:{10}}-----{spacy.explain(tok.pos_)} ')             #.tag/.pos will give the numeric reference, .tag_/.pos_ will give the actual name

The                 ----DET       -----determiner 
drunk               ----ADJ       -----adjective 
nigger              ----NOUN      -----noun 
drove               ----VERB      -----verb 
his                 ----PRON      -----pronoun 
car                 ----NOUN      -----noun 
into                ----ADP       -----adposition 
a                   ----DET       -----determiner 
crowd               ----NOUN      -----noun 
and                 ----CCONJ     -----coordinating conjunction 
killed              ----VERB      -----verb 
4                   ----NUM       -----numeral 
people              ----NOUN      -----noun 


In [22]:
for tok in doc:
    print(f'{tok.text:{20}}-----{tok.tag_:20}-----{spacy.explain(tok.tag_)} ')

The                 -----DT                  -----determiner 
drunk               -----JJ                  -----adjective (English), other noun-modifier (Chinese) 
nigger              -----NN                  -----noun, singular or mass 
drove               -----VBD                 -----verb, past tense 
his                 -----PRP$                -----pronoun, possessive 
car                 -----NN                  -----noun, singular or mass 
into                -----IN                  -----conjunction, subordinating or preposition 
a                   -----DT                  -----determiner 
crowd               -----NN                  -----noun, singular or mass 
and                 -----CC                  -----conjunction, coordinating 
killed              -----VBD                 -----verb, past tense 
4                   -----CD                  -----cardinal number 
people              -----NNS                 -----noun, plural 


In [23]:
pos_counts= doc.count_by(spacy.attrs.POS)

In [24]:
for (k,v) in pos_counts.items():
    print(f'{doc.vocab[k].text:{20}}-------{v} ')

DET                 -------2 
ADJ                 -------1 
NOUN                -------4 
VERB                -------2 
PRON                -------1 
ADP                 -------1 
CCONJ               -------1 
NUM                 -------1 


In [25]:
tag_counts= doc.count_by(spacy.attrs.TAG)

In [26]:
for (k,v) in tag_counts.items():
    tag= doc.vocab[k].text
    print(f'{spacy.explain(tag):{50}}--------{v} ')

determiner                                        --------2 
adjective (English), other noun-modifier (Chinese)--------1 
noun, singular or mass                            --------3 
verb, past tense                                  --------2 
pronoun, possessive                               --------1 
conjunction, subordinating or preposition         --------1 
conjunction, coordinating                         --------1 
cardinal number                                   --------1 
noun, plural                                      --------1 


In [27]:
dep_counts= doc.count_by(spacy.attrs.DEP)

In [28]:
for (k,v) in dep_counts.items():
    dep= doc.vocab[k].text
    print(f"{spacy.explain(dep)}---------{v} ")

determiner---------2 
adjectival modifier---------1 
nominal subject---------1 
None---------1 
possession modifier---------1 
direct object---------2 
prepositional modifier---------1 
object of preposition---------1 
coordinating conjunction---------1 
conjunct---------1 
numeric modifier---------1 


In [29]:
def show_ents(doc):
    if len(doc.ents) == 0:
        print("No entitiies found")
    else:
        for ent in doc.ents:
            print(f'{ent.text}-------{ent.label_}------{spacy.explain(ent.label_)} ')

In [30]:
show_ents(doc)

4-------CARDINAL------Numerals that do not fall under another type 


In [31]:
show_ents(nlp(u'Raneld bought a new Mazda RX7 with LS7 swapped for drifting'))

Raneld-------PERSON------People, including fictional 
Mazda RX7-------ORG------Companies, agencies, institutions, etc. 


In [32]:
doc1, doc2= nlp(u"I swapped a Hell_ inside my RX7."), nlp(u"Hell_ is my engine of choice for a track car")

In [33]:
show_ents(doc1)

RX7-------ORG------Companies, agencies, institutions, etc. 


In [34]:
show_ents(nlp(u"I swapped an LS7 engine inside my Celica AWD"))

LS7-------GPE------Countries, cities, states 
Celica-------PRODUCT------Objects, vehicles, foods, etc. (not services) 


In [35]:
from spacy.tokens import Span

In [36]:
#Here we are going to add "Hell_" inside the PRODUCT entity.  
#We couldnt have added "LS7" inside the PRODUCT entity cuz it already had an entity defined for it.
PRODUCT= doc1.vocab.strings[u"PRODUCT"]

In [37]:
PRODUCT

386

In [38]:
new_ent= Span(doc1, 3,4, label= PRODUCT)
type(new_ent)
    

spacy.tokens.span.Span

In [39]:
doc1.ents= tuple(list(doc1.ents)+[new_ent])

In [40]:
show_ents(doc1)

Hell-------PRODUCT------Objects, vehicles, foods, etc. (not services) 
RX7-------ORG------Companies, agencies, institutions, etc. 


In [41]:
doc_vac= nlp(u'This is the new vacuum cleaner in the market.'u'This is the vacuum-cleaner I will be using')

In [42]:
show_ents(doc_vac)

No entitiies found


In [43]:
from spacy.matcher import PhraseMatcher
matcher= PhraseMatcher(nlp.vocab)

In [44]:
phrase_list= ['vacuum cleaner', 'vacuum-cleaner']

In [45]:
phrase_pattern= [nlp(text) for text in phrase_list]

In [46]:
matcher.add('Vac_Clean', None, *phrase_pattern)

In [47]:
matches= matcher(doc_vac)

In [48]:
print(matches)

[(11464633557851095265, 4, 6), (11464633557851095265, 13, 16)]


In [49]:
#Now, adding the two different types of vacuum cleaner in the PRODUCT tag
PROD= doc_vac.vocab.strings[u'PRODUCT']

In [50]:
new_ents_vac= [Span(doc_vac, match[1], match[2]) for match in matches]
print(new_ents_vac)

[vacuum cleaner, vacuum-cleaner]


In [51]:
doc_vac.ents= tuple(list(doc_vac.ents) + new_ents_vac)
print(doc_vac.ents)

()


In [52]:
show_ents(doc_vac)

No entitiies found


In [53]:
len([ent for ent in doc_vac.ents if ent.label_=="PRODUCT"])         #For seeing the number of a particualr entity in a doc

0

In [54]:
type(doc_vac)

spacy.tokens.doc.Doc

In [55]:
for tok in doc_vac:
    print(tok)

This
is
the
new
vacuum
cleaner
in
the
market
.
This
is
the
vacuum
-
cleaner
I
will
be
using


In [56]:
doc_sent= nlp(u"This is the new vacuum cleaner in the market.This is the new vacuum cleaner in the market.This is the new vacuum cleaner in the market. ")

In [57]:
for sent in doc_sent.sents:
    print(sent)

This is the new vacuum cleaner in the market.
This is the new vacuum cleaner in the market.
This is the new vacuum cleaner in the market.


In [58]:
type(doc_sent.sents)

generator

In [59]:
print(list(doc_sent.sents)[0])
type(list(doc_sent.sents)[0])

This is the new vacuum cleaner in the market.


spacy.tokens.span.Span

In [60]:
doc_vac.text

'This is the new vacuum cleaner in the market.This is the vacuum-cleaner I will be using'

In [87]:
doc= nlp(u'"Your management is doing quiet well. Keep it up; hope we can double our revenue this year" -Dilip Singh')

In [78]:
doc.text

'"Your management is doing quiet well; hope we can double our revenue this year" -Dilip Singh'

In [88]:
for sent in doc.sents:
    print(sent)

"Your management is doing quiet well. Keep it up;
hope we can double our revenue this year" -Dilip Singh


In [80]:
#For correctly segmenting the above doc, we are creating a new SEGMENTATION RULE
from spacy.language import Language
@Language.component('set_custom_boundaries')
def set_custom_boundaries(doc):
    for token in doc[:-1]:
        if  token.text==";":
            doc[token.i+1].is_sent_start= True                  #token.i gives the index position.
        else:
            doc[token.i+1].is_sent_start= False
    return doc

In [93]:
nlp.add_pipe('set_custom_boundaries', before= 'parser')
nlp.pipe_names

['tok2vec',
 'tagger',
 'set_custom_boundaries',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ner']

In [94]:
nlp.remove_pipe('set_custom_boundaries')
print(nlp.pipe_names)

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [95]:
doc= nlp(doc.text)

In [96]:
for sent in doc.sents:
    print(sent)

"Your management is doing quiet well.
Keep it up; hope we can double our revenue this year" -Dilip Singh


In [107]:
doc1= nlp(u'Sentence 1. Sentence\n. sentence 2\nsex. \n')
doc2= nlp(u'This is a sent. This is another sent.\n\n. This is another')

In [70]:
for sent in doc2.sents:
    print(sent.text)

This is a sent.
This is another sent.


.
This is another


In [71]:
for w in doc2:
    print(f'{w.text}------{w.i} ')

This------0 
is------1 
a------2 
sent------3 
.------4 
This------5 
is------6 
another------7 
sent------8 
.------9 


------10 
.------11 
This------12 
is------13 
another------14 


In [76]:
'''def newline_segment(doc):                   #It segments the sentence based on new line '\n
    start=0
    new_line= False
    for word in doc:
        if new_line:
            yield doc[start:word.i]
            start= word.i
            new_line= False
        elif word.text=='\n':
            new_line= True
    yield doc[start:]'''
    

In [127]:
@Language.component('new_line_bound')
def new_line_bound(doc):
    for tok in doc[:-1]:
        if tok.text=='\n':
            doc[tok.i+1].is_sent_start= True
        else:
            doc[tok.i+1].is_sent_start= False
    return doc

In [129]:
nlp.add_pipe('new_line_bound', before= 'parser')
nlp.pipe_names

['tok2vec',
 'tagger',
 'new_line_bound',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ner']

In [128]:
nlp.remove_pipe('new_line_bound')
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [130]:
doc= nlp(doc1.text)

In [131]:
for sent in doc.sents:
    print(sent)

Sentence 1. Sentence

. sentence 2

sex. 

